<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup

# Конечно, с игнорированием предупреждений нужно быть осторожным,
# но при вызове запросов указанными в модуле методами возникают предупреждения
import warnings
warnings.filterwarnings('ignore') 

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 0

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
    Select count(id) as vac_count
    From vacancies    
'''

In [5]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
display(df)

,vac_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
    Select count(id) as empl_count
    From employers    
'''

In [7]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
display(df)

,empl_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
    Select count(id) as area_count
    From areas    
'''

In [9]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
display(df)

,area_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_3 = f'''
    Select count(id) as industr_count
    From industries    
'''

In [11]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
display(df)

,industr_count
0,294


***

In [12]:
# выводы по предварительному анализу данных

Количество вакансий примерно сопоставимо с количеством работодателей (в среднем около 2-3 вакансий на одного работодателя). Это говорит, что большого спроса на рынке вакансий нет. Пояление новых вакансий носит, если можно так выразиться, "*штучный*" характер. Возможно, это связано с замедлением развития экономики с соответствующим замедлением появления новых компаний. При большой доле новых компаний количество вакансий приходящихся в среднем на одну компаниюю должно быть больше. По регионам аналогично, грубо говоря, 20 000 компаний на 1 000 регионов уж точно не говорит о бурном экономическом росте. Аналогично с отраслями 20 000 компаний на 300 отраслей тоже говорит о том, что на рынке труда предложений немного и условия задают работодатели. 50 000 вакансий это немного для населения России.
Говорить о том что, данные hh.ru могут не сооветствовать реальной ситуации на рынке труда, в наше время интернета, мне кажется, уже неуместно.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query_4_1 = f'''
    Select a.name,count(v.id) as cnt -- count по id надежней (априори нет незаполненных значений)
    From areas as a
        Join vacancies as v On v.area_id = a.id --соединяем таблицы вакансий с таблицей регионов
    Group by a.name -- Группируем по имени региона
    Order by count(v.id) Desc -- Сортировка по количеству вакансий по убыванию
'''

In [14]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
display(df)

,name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query_4_2 = f'''
    Select count(id) as count_fork
    From vacancies
    Where salary_from is not null or salary_to is not null
'''

In [16]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
display(df)

,count_fork
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query_4_3 = f'''

    /* Решил сделать вывод через объедение двух запросов*/
    /* Среднее значение определяется обычным делением суммы значений на количество значений */
    
    (
        Select ROUND( SUM( ROUND(salary_from) ) / COUNT(salary_from) ) as "Значение зарплаты, руб", 'Нижняя граница' as "Описание"
        From vacancies
        Where salary_from Is Not Null or salary_to Is Not Null
    )
    UNION ALL
    (
        Select ROUND( SUM( ROUND(salary_to) ) / COUNT(salary_to) ), 'Верхняя граница'
        From vacancies
        Where salary_from Is Not Null or salary_to Is Not Null
    )
    
'''

In [18]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
display(df)

,"Значение зарплаты, руб",Описание
0,71065.0,Нижняя граница
1,110537.0,Верхняя граница


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query_4_4 = f'''

    -- Использую соединение строк через ||
    -- Группировка сначала по рабочему графику затем по типу трудоустройства
    -- В резельтате будут пары schedule — employment 
        
    Select schedule||' — '||employment as sched_empl, COUNT(id) as count_sched_empl
    From vacancies
    Group by schedule,employment
    Order by COUNT(id) Desc
'''

In [20]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
display(df)

,sched_empl,count_sched_empl
0,Полный день — Полная занятость,35367
1,Удаленная работа — Полная занятость,7802
2,Гибкий график — Полная занятость,1593
3,Удаленная работа — Частичная занятость,1312
4,Сменный график — Полная занятость,940
5,Полный день — Стажировка,569
6,Вахтовый метод — Полная занятость,367
7,Полный день — Частичная занятость,347
8,Гибкий график — Частичная занятость,312
9,Полный день — Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query_4_5 = f'''
    Select experience, COUNT(id)
    From vacancies    
    Group by experience
    Order by COUNT(id)
'''

In [22]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
display(df)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [23]:
# выводы по детальному анализу вакансий

Распределение вакансий по регионам, в целом, пропорционально населению этих регионов. Но я в конце проекта сделаю для городов милионников, таблицу с отношением количества вакансий к населению городов, тогда возможно, Москва уже не будет на первом месте. По опыту работы отмечу,что достаточно большая часть работодателей могут взять на вакансию "Без опыта", скорей всего это работа не требующая большой подготовки. Почти всем требуются сотрудники с полной занятостью, и совсем немногие готовы взять на стажировку. Большая часть работодателей готова взять на удалённую работу, тут опять же играет роль современная доступность интернета и, как показало время Covid, понимание работодателями того, что удалённая работа может быть такой же эффективной, как и работа в офисе. 

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [24]:
# текст запроса
query_5_1 = f'''

    -- несколько топорно, но сделал два одинаковых запроса через Union
    -- Отсортировал по убыванию количества вакансий
    -- и с помощью Offset, Limit вывел нужные места ))

    (
        Select e.name, COUNT(v.id)
            From vacancies as v
                Join employers as e On v.employer_id = e.id -- соединение таблицы вакансий с таблицей работодателей             
        Group by e.id
        Order by COUNT(v.id) Desc
        Limit 1
    ) 
    UNION ALL
    (
        Select e.name, COUNT(v.id)
            From vacancies as v
                Join employers as e On v.employer_id = e.id                
        Group by e.id
        Order by COUNT(v.id) Desc
        Offset 4
        Limit 1
    )  
'''

In [25]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
display(df)

,name,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [26]:
# текст запроса
query_5_2 = f'''

/*
    для сохранения регионов, в которых нет вакансий использовал Left Join
    сделал отдельные запросы с выводом регионов и количества работодателей (группировка по регионам)
    и выводом регионов и количества вакансий (группировка по регионам)
    затем уже результаты этих двух запросов соединил между собой
    конечная сортировка по условию задания
    конечно, всегда смотрю контрольные точки
    количество строк должно совпадать с  количеством регинов задание 3_3
*/

With a_e as
(
    Select a.id as id, a.name as name,  COUNT(e.id) as empl_count
        From areas as a
        Left Join employers as e On a.id = e.area      
    Group by a.id   
),
a_v as
(
    Select a.id as id, COUNT(v.id) as vac_count
        From areas as a
        Left Join vacancies as v On a.id = v.area_id      
    Group by a.id
)

Select ae.name, ae.empl_count, av.vac_count
    From a_e as ae
        Left Join a_v as av On ae.id = av.id
Order by av.vac_count, ae.empl_count Desc     
'''

In [27]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
display(df)

,name,empl_count,vac_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [28]:
# текст запроса
query_5_3 = f'''

    -- Соединение слева двух таблиц работодателей и вакансий, при этом подсчёт количества по area_id
    -- Использовал DISTINCT, так как у работодателя может быть несколько вавкансий в одном регионе
    -- количестов строк должно совпадать с заданием 3_2
    
    Select e.name, COUNT( DISTINCT v.area_id )
        From employers as e
            Left Join vacancies as v On e.id = v.employer_id
    Group by e.id
    Order by count(v.area_id) Desc
'''

In [29]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
display(df)

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Тинькофф,43
3,СБЕР,24
4,Газпром нефть,24
...,...,...
23496,МАКМАХ,0
23497,Завод Стелкон,0
23498,ИнтерЭВМ,0
23499,Фёдоров Михаил,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
# текст запроса
query_5_4 = f'''

    -- соединил слева таблицу работодателей с таблицей пар ключей
    -- и вывел работодалей, которых нет в этой таблице
    
    Select Count(e.id)
        From employers as e
            Left Join employers_industries as ei On e.id = ei.employer_id
    Where  ei.employer_id is Null    
'''

In [31]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
display(df)

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
# текст запроса
query_5_5 = f'''
    Select e.name, Count(ei.industry_id)
        From employers as e
            Join employers_industries as ei On e.id = ei.employer_id
    Group by e.id
    Having Count(ei.industry_id) = 4
    Order by e.name
    Offset 2
    Limit 1
''' 

In [33]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
display(df)

,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [34]:
# текст запроса
query_5_6 = f'''
    Select Count(e.id)
        From employers as e
            Join employers_industries as ei On e.id = ei.employer_id
                Join industries as i On ei.industry_id = i.id And i.name = 'Разработка программного обеспечения'
''' 

In [35]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
display(df)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим заданием посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [36]:
# код для получения списка городов-милионников
# сделал после изучения кода страницы в wikipedia
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
cities_list = []
# Цикл по тегам tr в таблице
for i , tr in enumerate(page.findAll('tbody')[1]): # Таблица со списком городов-миллионников это вторая таблица
    if i == 0:
        continue # Первый title в таблице не имеет отношения к названиям городов-миллионников
    # Извлекаем из тега a аттрибут title и доабвляем к списку, с учетом
    # формирования будущих запросов
    cities_list.append("'" + tr.find('a')['title'] + "'")
cities = ','.join(cities_list)

In [37]:
# текст запроса
query_5_7 = f'''

    /*

        Все запросы написаны с использованием f-строки, поэтому можно вставить в запрос
        полученный ранее список горожов-милионников cities
        объединяем два запроса один сггруппированный по имени регина
        второй общее количество поэтому без группировки
        соединяем три таблицы: работодатели, вакансии и регионы
        
    */
    
    (
        Select a.name as name, COUNT(v.id) as cnt
            From employers as e
                Join vacancies as v On e.id = v.employer_id
                    Join areas as a On v.area_id = a.id 
        Where e.name = 'Яндекс' and a.name in ({cities})
        Group by a.name
    )
    UNION ALL
    (
        Select 'Total', COUNT(v.id)
            From employers as e
                Join vacancies as v On e.id = v.employer_id
                    Join areas as a On v.area_id = a.id 
        Where e.name = 'Яндекс' and a.name in ({cities})
    )
    Order by cnt
''' 


In [38]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
display(df)

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


In [39]:
# выводы по анализу работодателей

Конечно, обращает на себя внимание большое количество вакансий в Яндекс. Это говорит о значительном развитии компании. Теперь про вакансии в регионах. Ряд компаний (например, СБЕР) размещают вакансии  в регионах, так как имеют филиалы в регионах и им нужны люди из конкрентых регионов для работы в конретных филиалах. Другие компании (например, Яндекс), возможно, просто ищут в регионах новые кадры и так как эти люди будут скорей всего работать удалённо, то неважно из какого региона они будут. Если говорить про количество работодателей, у которых в качестве сферы деятельности указана *Разработка программного обеспечения*, то оно относительно небольшое. Лучше, возможно, увидеть сколько вакансий у каждой из этих 3500 компаний. Такую таблицу я подготовлю в конце проекта.

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [40]:
# текст запроса
query_6_1 = f'''
    Select Count(id)
        From vacancies
    Where 
        lower(name) Like '%data%'
        Or
        lower(name) Like '%данн%'
''' 

In [41]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
display(df)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [42]:
# текст запроса
query_6_2 = f'''
    Select Count(id)
        From vacancies
    Where 
            (
                    lower(name) Like '%data scientist%'
                Or
                    lower(name) Like '%data science%'
                Or
                    lower(name) Like '%исследователь данных%'
                Or
                    lower(name) Like '%machine learning%'
                Or
                    lower(name) Like '%машинн%обучен%'
                Or
                    (
                            name Like '%ML%'
                        And
                            name Not Like '%HTML%'            
                    )    
            )

        And
            (
                    lower(name) Like '%junior%'
                Or
                    lower(experience) Like '%нет опыта%'
                Or
                    lower(employment) Like '%cтажировка%'
            )
'''

In [43]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
display(df)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [44]:
# текст запроса
query_6_3 = f'''
    Select Count(id)
        From vacancies
    Where 
            (
                    lower(name) Like '%data scientist%'
                Or
                    lower(name) Like '%data science%'
                Or
                    lower(name) Like '%исследователь данных%'
                Or
                    lower(name) Like '%machine learning%'
                Or
                    lower(name) Like '%машинн%обучен%'
                Or
                    (
                            name Like '%ML%'
                        And
                            name Not Like '%HTML%'            
                    )    
            )

    /*    And Not
            (
                    lower(name) Like '%junior%'
                Or
                    lower(experience) Like '%нет опыта%'
                Or
                    lower(employment) Like '%cтажировка%'
            )   */
        And
            (
                    lower(key_skills) like '%sql%'
                Or
                    lower(key_skills) like '%postgres%'
            )
'''

In [45]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
display(df)

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [46]:
# текст запроса
query_6_4 = f'''
    Select Count(id)
        From vacancies
    Where 
            (
                    lower(name) Like '%data scientist%'
                Or
                    lower(name) Like '%data science%'
                Or
                    lower(name) Like '%исследователь данных%'
                Or
                    lower(name) Like '%machine learning%'
                Or
                    lower(name) Like '%машинн%обучен%'
                Or
                    (
                            name Like '%ML%'
                        And
                            name Not Like '%HTML%'            
                    )    
            )

        And

            (
                    lower(key_skills) like '%python%'
            )
'''


In [47]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
display(df)

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [48]:
# текст запроса
query_6_5 = f'''

    SELECT ROUND(  SUM( array_length ( string_to_array(key_skills, '\t') , 1) ) / ( 1.0 * Count(key_skills) ) , 2) 
        From vacancies
     Where 
            (
                    lower(name) Like '%data scientist%'
                Or
                    lower(name) Like '%data science%'
                Or
                    lower(name) Like '%исследователь данных%'
                Or
                    lower(name) Like '%machine learning%'
                Or
                    lower(name) Like '%машинн%обучен%'
                Or
                    (
                            name Like '%ML%'
                        And
                            name Not Like '%HTML%'            
                    )    
            )
        And
            (
                key_skills is not null
            )

            
'''

In [49]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
display(df)

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [50]:
# текст запроса
query_6_6 = f'''

    With avs as
    (
        (
            Select *, ( salary_from + salary_to ) / 2.0 as aver_sal
                From vacancies 
            Where 
                    salary_from Is Not Null
                And
                    salary_to Is Not Null
        )
        UNION ALL
        (
            Select *, coalesce(salary_from  , 0) * 1.0 + coalesce(salary_to , 0) * 1.0 as aver_sal
                From vacancies 
            Where
                (
                    (     
                            salary_from Is Not Null
                        And
                            salary_to Is Null
                    )
                    Or
                        (     
                            salary_from Is Null
                        And
                            salary_to Is Not Null
                    )                   
                )
        )
    )
    
    Select experience, ROUND( SUM(aver_sal) / COUNT (id) , 0)
        From avs
        
    Where 
        (
                lower(name) Like '%data scientist%'
            Or
                lower(name) Like '%data science%'
            Or
                lower(name) Like '%исследователь данных%'
            Or
                lower(name) Like '%machine learning%'
            Or
                lower(name) Like '%машинн%обучен%'
            Or
                (
                        name Like '%ML%'
                    And
                        name Not Like '%HTML%'            
                )    
        )
        
    Group by experience
    Order by  ( SUM(aver_sal) / COUNT (id) )
            
'''

In [51]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
display(df)

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [52]:
# выводы по предметному анализу

Количество вакансий для работы с данными примерно соответствует вакансиям Яндекса. В конце проекта сделаю таблицу с количеством вакансий для работы с *данными* по работодателям. Начинающие дата-сайентисты особо никому не нужны, всего около 50 вакансий. У дата-сайентистов должно быть около 6 разных навыков, это объяснимо, так как может требоваться различный анализ разных данных и для этого необходимо уметь пользоваться разными инструментами, что компенсируется уровнем зарплаты. 

# Общий вывод по проекту

In [53]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

Приятно удивило значительное количество вакансий у Яндекса. Компания развивается. Версия, сделанная мною в конце 6 модуля, про совпадение количества вакансий Яндекса с количеством вакансий для работы с данными абсолютно не подтвердилась (см. запрос ниже 7_3). Конечно, не хватает временного сравнения сделанных запросов. То есть сравнение одних и тех же таблиц за разные периоды времени (ежегодно). Тогда будет понятна, допустим, динамика вакансий дата-сайентистов. Но очевмдно, что раньше таких вакансий просто не было, как и не было такого понятия. Здесь же уже около 500 вакансий (см. запрос ниже 7_4). Если сравнить таблицу 7_4 и 7_2, то видно, что бакнковский сектор достаточно активно ищет сотрудников для работы с данными. IT уже давно неотъемлемая и значительная часть в банковской деятельности.

Ниже сделал два dataframe, с помощью которых получил таблицу с количеством вакансий на тысячу жителей городов-милионниках. Москва оказалась на последнем месте. На первом месте Нижний Новгород. Это, в частности, говорит о разном росте экономики в городах-милионниках. Для dataframe с населением городов милионников также сделал таблицу. 

In [54]:
# код для получения списка городов-милионников с населением
# сделал после изучения кода страницы в wikipedia
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
cities_popul = list([]) # создание списка списков
# Цикл по тегам tr в таблице
for i , tr in enumerate(page.findAll('tbody')[1]): # Таблица со списком городов-миллионников это вторая таблица
    if i == 0:
        continue # Первый title в таблице не имеет отношения к названиям городов-миллионников
    city = tr.find('a')['title'] # получаем название города-милионника
    for j, td in  enumerate(tr):
       if j !=2:
            continue
       popul = td['data-sort-value'] # получаем численность населения
    cities_popul.append([city, popul]) # добавляем город и его население как список в список таких пар

# На соновании полученного списка списков создаём datafrane
df_city_popul = pd.DataFrame(cities_popul, columns =['city', 'population'])
print(df_city_popul) 
    # Извлекаем из тега a аттрибут title и доабвляем к списку, с учетом
    # формирования будущих запросов

               city population
0            Москва      13098
1   Санкт-Петербург       5598
2       Новосибирск       1635
3      Екатеринбург       1539
4            Казань       1315
5   Нижний Новгород       1213
6        Красноярск       1197
7         Челябинск       1183
8            Самара       1164
9               Уфа       1158
10   Ростов-на-Дону       1136
11        Краснодар       1121
12             Омск       1111
13          Воронеж       1052
14            Пермь       1027
15        Волгоград       1026


In [55]:
# Запрос для получения количества вакансий в городах-милионниках
query_7_1 = f'''

    (
        Select a.name as name, COUNT(v.id) as "count"
            From vacancies as v
                Join areas as a On v.area_id = a.id 
        Where  a.name in ({cities})
        Group by a.name
    )
    
'''

In [56]:
# query_7_1
# результат запроса
df = pd.read_sql_query(query_7_1, connection)

# Соединение (по городу) dataframe с количеством вакансий по городам милионикам
# с dataframe с населением по городам-милионникам
df = df.merge(    
    df_city_popul,
    left_on=['name'],
    right_on=['city']
    )


df[['count', 'population']] = df[['count', 'population']].astype(float)
df['vacancies/population'] = df['count'] / df['population']

# расчет количества вакансий на тысячу жителей
df['vacancies/population'] = df['vacancies/population'].round(2)

df = df.drop(columns = ['name','count'])
df = df.sort_values('vacancies/population',ascending=False) # сортировка по убыванию

df.reset_index(drop=True, inplace=True) # "перезагрузка" индексов

display(df)

,city,population,vacancies/population
0,Нижний Новгород,1213.0,1.38
1,Новосибирск,1635.0,1.23
2,Краснодар,1121.0,1.16
3,Екатеринбург,1539.0,1.10
4,Казань,1315.0,1.08
5,Воронеж,1052.0,1.01
6,Ростов-на-Дону,1136.0,1.00
7,Самара,1164.0,0.98
8,Пермь,1027.0,0.75
9,Красноярск,1197.0,0.71


In [57]:
# Запрос для определения количества вакансий работодателей, 
# у которых в качестве сферы деятельности указана Разработка программного обеспечения
query_7_2 = f'''
    Select e.name, Count(v.id)
        From vacancies as v
            Join employers as e On v.employer_id = e.id
                Join employers_industries as ei On e.id = ei.employer_id
                    Join industries as i On ei.industry_id = i.id 
    Where i.name = 'Разработка программного обеспечения'
    Group by e.name
    Order by Count(v.id) Desc
'''

In [58]:
# результат запроса
df = pd.read_sql_query(query_7_2, connection)
display(df)

,name,count
0,Тинькофф,444
1,DataArt,247
2,Совкомбанк Технологии,204
3,IBS,175
4,Тензор,155
...,...,...
2579,Индустрия делового программного обеспечения,1
2580,"Айтекс, Центр Информационных Технологий",1
2581,Айтек,1
2582,"ИНКОМ, группа компаний",1


In [59]:
# текст для вывода таблицы с количеством вакансий для работы с ДАННЫМИ по работодателям
# текст запроса
query_7_3 = f'''
    Select e.name, Count(v.id)
        From vacancies as v
            Join employers as e On v.employer_id = e.id 
    Where 
        lower(v.name) Like '%data%'
        Or
        lower(v.name) Like '%данн%'
    
    Group by e.id
    Order by Count(v.id) Desc
'''

In [60]:
# результат запроса
df = pd.read_sql_query(query_7_3, connection)
display(df)

,name,count
0,СБЕР,100
1,DataArt,46
2,МТС,40
3,Банк ВТБ (ПАО),31
4,Тинькофф,22
...,...,...
727,"People group, центр подбора и развития персонала",1
728,Открытый код,1
729,Сима-ленд,1
730,ЭР-Телеком,1


In [61]:
# текст запроса для определения количества вакансий дата-сайентистов
# просто ранее потроенный запрос 6_2, но без выделения junior
query_7_4 = f'''
    Select Count(id)
        From vacancies
    Where 
            (
                    lower(name) Like '%data scientist%'
                Or
                    lower(name) Like '%data science%'
                Or
                    lower(name) Like '%исследователь данных%'
                Or
                    lower(name) Like '%machine learning%'
                Or
                    lower(name) Like '%машинн%обучен%'
                Or
                    (
                            name Like '%ML%'
                        And
                            name Not Like '%HTML%'            
                    )    
            )
'''

In [62]:
# результат запроса
df = pd.read_sql_query(query_7_4, connection)
display(df)

,count
0,480


In [63]:
# не забываем закрыть соединение после окончания работы
connection.close()